In [3]:
# -*- coding: utf-8 -*-
# Generated by the protocol buffer compiler.  DO NOT EDIT!
# source: chess.proto
# Protobuf Python Version: 4.25.1
"""Generated protocol buffer code."""
from google.protobuf import descriptor as _descriptor
from google.protobuf import descriptor_pool as _descriptor_pool
from google.protobuf import symbol_database as _symbol_database
from google.protobuf.internal import builder as _builder
# @@protoc_insertion_point(imports)

_sym_db = _symbol_database.Default()




DESCRIPTOR = _descriptor_pool.Default().AddSerializedFile(b'\n\x0b\x63hess.proto\"\x1a\n\tChessGame\x12\r\n\x05moves\x18\x01 \x03(\t\")\n\x0c\x43hessDataSet\x12\x19\n\x05games\x18\x01 \x03(\x0b\x32\n.ChessGameb\x06proto3')

_globals = globals()
_builder.BuildMessageAndEnumDescriptors(DESCRIPTOR, _globals)
_builder.BuildTopDescriptorsAndMessages(DESCRIPTOR, 'chess_pb2', _globals)
if _descriptor._USE_C_DESCRIPTORS == False:
  DESCRIPTOR._options = None
  _globals['_CHESSGAME']._serialized_start=15
  _globals['_CHESSGAME']._serialized_end=41
  _globals['_CHESSDATASET']._serialized_start=43
  _globals['_CHESSDATASET']._serialized_end=84
# @@protoc_insertion_point(module_scope)


In [2]:
!pip install protobuf

  Obtaining dependency information for protobuf from https://files.pythonhosted.org/packages/e6/db/7b2edc72807d45d72f9db42f3eb86ddaf37f9e55d923159b1dbfc9d835bc/protobuf-4.25.1-cp37-abi3-macosx_10_9_universal2.whl.metadata
Using cached protobuf-4.25.1-cp37-abi3-macosx_10_9_universal2.whl (394 kB)

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [ ]:
import torch
import chess
from torch.utils.data import Dataset

class ChessDataset(Dataset):
    def __init__(self, games):
        self.positions = torch.Tensor
        self.moves = torch.Tensor
        self.process_games(games)

    def process_games(self, games: list[list[str]]) -> None:
        pos = []
        movs = []
        
        for game in games:
            board = chess.Board()
            for move_san in game:
                try:
                    move = board.parse_san(move_san)
                    if board.is_legal(move):
                        # Add current board state to positions
                        pos.append(self.board_to_tensor(board))
                        # Make move and add to moves
                        movs.append(self.move_to_tensor(move))
                        board.push(move)
                except ValueError:
                    # Handle errors or illegal moves
                    break
        
        # Convert lists to PyTorch tensors
        self.positions = torch.stack(pos)
        self.moves = torch.stack(movs)
                    
    def board_to_tensor(self, board: chess.Board) -> torch.Tensor:
        # Initialize the tensor with shape (12, 8, 8)
        tensor = torch.zeros((12, 8, 8), dtype=torch.float)

        piece_idx = {'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
                    'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11}

        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece:
                # Get the piece type
                piece_type = piece.symbol()
                row, col = divmod(square, 8)
                # Populate the tensor
                tensor[piece_idx[piece_type], row, col] = 1

        return tensor


    def move_to_tensor(self, move: chess.Move) -> torch.Tensor:
        # Initialize a one-hot encoded tensor of length 4096
        one_hot_move = torch.zeros(64 * 64, dtype=torch.float)

        # Calculate the index for the one-hot vector
        source = move.from_square
        destination = move.to_square
        # We multiply the source by 64, given that for each source there are 64 possible moves
        # (the tensor dimensions assumes that a move can have the same source and destination - though
        # the network will learn that those moves are illegal)
        index = source * 64 + destination

        # Set the appropriate position to 1
        one_hot_move[index] = 1

        return one_hot_move

    def check_layout(self, index: int):
        ## Checking the positions input dataset
        game_layout = self.positions[index]

        for piece_type_coordinates in game_layout:
            print(piece_type_coordinates)

    def __len__(self):
        return len(self.positions)

    def __getitem__(self, idx):
        return self.positions[idx], self.moves[idx]